In [ ]:
# Practice: Configuring SSH Security Options

# bind sshd to both port 20 and port 2022, so that you can still open an ssh session if you made a config error 
# open two ssh session when config ssh. User one session to apply changes and test, user the other one to keep the current connection option. 
# After the config change, reload/restart the sshd service. Ative session will not be disconnected after restarting the ssh server.

# this is done from the remote host ssh session
[root@localhost ssh]# pwd
/etc/ssh
[root@localhost ssh]# cat sshd_config
# If you want to change the port on a SELinux system, you have to tell
# SELinux about this change.
# semanage port -a -t ssh_port_t -p tcp #PORTNUMBER
#
#Port 22
#AddressFamily any
#ListenAddress 0.0.0.0
#ListenAddress ::

# add port 2022, allow user lisa
[root@localhost ssh]# vi sshd_config
# If you want to change the port on a SELinux system, you have to tell
# SELinux about this change.
# semanage port -a -t ssh_port_t -p tcp #PORTNUMBER
#
Port 22
Port 2022
#AddressFamily any
#ListenAddress 0.0.0.0
#ListenAddress ::
AllowUsers lisa



In [ ]:
# restert sshd service to save the config change
[root@localhost ssh]# systemctl restart sshd

# bind to port 2022 failed, permission denied 
[root@localhost ssh]# systemctl status -l sshd
● sshd.service - OpenSSH server daemon
   Loaded: loaded (/usr/lib/systemd/system/sshd.service; enabled; vendor preset>
   Active: active (running) since Tue 2020-10-06 00:04:55 MDT; 1min 4s ago
     Docs: man:sshd(8)
           man:sshd_config(5)
 Main PID: 9309 (sshd)
    Tasks: 1 (limit: 11154)
   Memory: 1.4M
   CGroup: /system.slice/sshd.service
           └─9309 /usr/sbin/sshd -D -oCiphers=aes256-gcm@openssh.com,chacha20-p>

Oct 06 00:04:55 localhost.localdomain systemd[1]: Stopped OpenSSH server daemon.
Oct 06 00:04:55 localhost.localdomain systemd[1]: Starting OpenSSH server daemo>
Oct 06 00:04:55 localhost.localdomain sshd[9309]: error: Bind to port 2022 on 0>
Oct 06 00:04:55 localhost.localdomain sshd[9309]: error: Bind to port 2022 on :>
Oct 06 00:04:55 localhost.localdomain systemd[1]: Started OpenSSH server daemon.
Oct 06 00:04:55 localhost.localdomain sshd[9309]: Server listening on 0.0.0.0 p>
Oct 06 00:04:55 localhost.localdomain sshd[9309]: Server listening on :: port 2>
lines 1-18/18 (END)...skipping...
● sshd.service - OpenSSH server daemon
   Loaded: loaded (/usr/lib/systemd/system/sshd.service; enabled; vendor preset: enabled)
   Active: active (running) since Tue 2020-10-06 00:04:55 MDT; 1min 4s ago
     Docs: man:sshd(8)
           man:sshd_config(5)
 Main PID: 9309 (sshd)
    Tasks: 1 (limit: 11154)
   Memory: 1.4M
   CGroup: /system.slice/sshd.service
           └─9309 /usr/sbin/sshd -D -oCiphers=aes256-gcm@openssh.com,chacha20-poly1305@openssh.com,aes256-ctr,aes256-cbc,aes128-gcm@openssh.com,aes128-ctr,aes128-cbc -oMACs=hmac-sha2-2>

Oct 06 00:04:55 localhost.localdomain systemd[1]: Stopped OpenSSH server daemon.
Oct 06 00:04:55 localhost.localdomain systemd[1]: Starting OpenSSH server daemon...
Oct 06 00:04:55 localhost.localdomain sshd[9309]: error: Bind to port 2022 on 0.0.0.0 failed: Permission denied.
Oct 06 00:04:55 localhost.localdomain sshd[9309]: error: Bind to port 2022 on :: failed: Permission denied.
Oct 06 00:04:55 localhost.localdomain systemd[1]: Started OpenSSH server daemon.
Oct 06 00:04:55 localhost.localdomain sshd[9309]: Server listening on 0.0.0.0 port 22.
Oct 06 00:04:55 localhost.localdomain sshd[9309]: Server listening on :: port 22.

In [ ]:
# modify SELinux security label for sshd port
[root@localhost ssh]# semanage port -l | grep ssh
ssh_port_t                     tcp      22

# add port 2022 for access to sshd
[root@localhost ssh]# semanage port -a -t ssh_port_t -p tcp 2022

[root@localhost ssh]# semanage port -l | grep ssh
ssh_port_t                     tcp      2022, 22

In [ ]:
# open the firewall for port 2022
[root@localhost ssh]# firewall-cmd --add-port=2022/tcp
success
[root@localhost ssh]# firewall-cmd --add-port=2022/tcp --permanent
success

In [ ]:
# restart sshd service, now sshd service is listening on two ports
[root@localhost ssh]# systemctl restart sshd

[root@localhost ssh]# systemctl status -l sshd
● sshd.service - OpenSSH server daemon
   Loaded: loaded (/usr/lib/systemd/system/sshd.service; enabled; vendor preset: enabled)
   Active: active (running) since Tue 2020-10-06 00:13:57 MDT; 1s ago
     Docs: man:sshd(8)
           man:sshd_config(5)
 Main PID: 9421 (sshd)
    Tasks: 1 (limit: 11154)
   Memory: 1.8M
   CGroup: /system.slice/sshd.service
           └─9421 /usr/sbin/sshd -D -oCiphers=aes256-gcm@openssh.com,chacha20-poly1305@openssh.com,aes256-ctr,aes256-cbc,aes128-gcm@openssh.com,aes128-ctr,aes128-cbc -oMACs=hmac-sha2-2>

Oct 06 00:13:57 localhost.localdomain systemd[1]: Stopped OpenSSH server daemon.
Oct 06 00:13:57 localhost.localdomain systemd[1]: Starting OpenSSH server daemon...
Oct 06 00:13:57 localhost.localdomain sshd[9421]: Server listening on 0.0.0.0 port 2022.
Oct 06 00:13:57 localhost.localdomain sshd[9421]: Server listening on :: port 2022.
Oct 06 00:13:57 localhost.localdomain sshd[9421]: Server listening on 0.0.0.0 port 22.
Oct 06 00:13:57 localhost.localdomain sshd[9421]: Server listening on :: port 22.
Oct 06 00:13:57 localhost.localdomain systemd[1]: Started OpenSSH server daemon.

# try to log in to the SSH server from the new port, test root access
[root@localhost ssh]# ssh -p 2022 lisa@localhost
The authenticity of host '[localhost]:2022 ([::1]:2022)' can't be established.
ECDSA key fingerprint is SHA256:4bfNoUfqjDgZWvfAq1Kkb4RrAWQrMUIUuP+gUZ620sM.
Are you sure you want to continue connecting (yes/no/[fingerprint])? yes
Warning: Permanently added '[localhost]:2022' (ECDSA) to the list of known hosts.
lisa@localhost's password:
Activate the web console with: systemctl enable --now cockpit.socket

Last login: Mon Oct  5 23:57:20 2020 from 192.168.219.1
[lisa@localhost ~]$ su -
Password:
[root@localhost ~]#